In [99]:
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from dataset_io import *
import numpy as np
from joblib import dump, load
import timeit

from sklearn.metrics import accuracy_score, f1_score

In [100]:
pallet_X=np.load(PROJECT_PATH+"pallet_rectangles_channels.npy")
background_X=np.load(PROJECT_PATH+"background_rectangles_channels.npy")

pallet_X=[np.dstack(x) for x in pallet_X]
background_X=[np.dstack(x) for x in background_X]

print(np.array(pallet_X).shape)
print(np.array(background_X).shape)

pallet_X=[x.flatten() for x in pallet_X]
background_X=[x.flatten() for x in background_X]

pallet_y=np.ones(len(pallet_X))
background_y=np.zeros(len(background_X))

X=np.vstack((pallet_X,background_X))
y=np.hstack((pallet_y,background_y))

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)
sss.get_n_splits(X, y)
train_index, test_index = list(sss.split(X, y))[0]
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

(2591, 24, 120, 8)
(7170, 24, 120, 8)


In [101]:
clf = RandomForestClassifier(128, n_jobs=-1) #128 is as fast as lower sizes and as accurate as greater sizes
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

acc=accuracy_score(y_test, y_pred)
f1=f1_score(y_test, y_pred)

print("acc: ",acc," f1: ",f1)

acc:  0.99795151928986  f1:  0.9961489088575095


In [102]:
dump(clf, 'rand_forest_clf.joblib') 

['rand_forest_clf.joblib']

### Compare original and dumped classifier speed

In [103]:
def predict_test():
    y_pred=clf.predict(X_test[:1])

timeit.timeit(predict_test, number=50)

5.3880882159992325

In [104]:
clf_load = load('rand_forest_clf.joblib') 

def predict_test_load():
    y_pred=clf_load.predict(X_test[:1])

timeit.timeit(predict_test_load, number=50)

5.390514238000833